### Use template code to load stored NewsAPI data from S3
### Replace variables with your own where is say "key-here"
### Do not post notebooks with keys to GitHub

In [1]:
import os.path
from os import path
import pandas as pd
import json
import boto3 
import os
from io import StringIO

import psycopg2
import pandas as pds
from sqlalchemy import create_engine

In [2]:
# API Key for News API
secret = 'key here'

In [3]:
# s3 secret - Carlos Keys (don't commit to GitHub)
ACCESS_KEY = "key-here" 
SECRET_KEY = "key-here" 
BUCKET_NAME ='key-here'

In [4]:
# S3 credentials
s3 = boto3.resource(
    's3',
    region_name='us-east-1',
    aws_access_key_id=ACCESS_KEY,
    aws_secret_access_key=SECRET_KEY
).Bucket(BUCKET_NAME)

json.load_s3 = lambda f: json.load(s3.Object(key=f).get()["Body"])
json.dump_s3 = lambda obj, f: s3.Object(key=f).put(Body=json.dumps(obj))

In [5]:
categories = ['business','entertainment','general','health','science','sports','technology']

## Json files stored
* resources/business_top_headline_data_new.json
* resources/entertainment_top_headline_data_new.json
* resources/general_top_headline_data_new.json
* resources/health_top_headline_data_new.json
* resources/science_top_headline_data_new.json
* resources/sports_top_headline_data_new.json
* resources/technology_top_headline_data_new.json

In [6]:
# Pull data
count =0
first = True
for category in categories:
    name = category
    fileName= f'resources/{category}_top_headline_data_new.json'
    json_buffer=json.load_s3('Project3/'+fileName)
    if first:
        category_dataframe=pd.read_json(json_buffer).T
        first = False 
    else :
        df = pd.read_json(json_buffer).T
        category_dataframe = category_dataframe.append(df,ignore_index=True)
    
print(category_dataframe.shape)
category_dataframe.category.value_counts()

(6513, 11)


sports           1133
business         1124
health           1023
entertainment     919
general           915
technology        911
science           488
Name: category, dtype: int64

In [7]:
# Save to Postgres (replace the variables, push to github)
jdbcUrl = ''


In [8]:
alchemyEngine   = create_engine(jdbcUrl, pool_recycle=3600);

In [9]:
postgreSQLConnection    = alchemyEngine.connect();

In [10]:
category_dataframe['publishedAt']= pd.to_datetime(category_dataframe['publishedAt'])

In [ ]:
postgreSQLTable= "sentiment_results"  
frame= category_dataframe[['author', 'title', 'description', 'url',
       'urlToImage', 'publishedAt', 'articleSummary', 'articleSentiment',
       'category']].to_sql(postgreSQLTable, postgreSQLConnection, if_exists='replace');